In [1]:
%env CUDA_VISIBLE_DEVICES=2
%env TRANSFORMERS_CACHE=/mnt/LLM/hub
%env OMP_NUM_THREADS=16

import os
import sys
sys.path.insert(0, '..')

import time
import random
from tqdm.auto import trange
import ipynbname  # pip install ipynbname

import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers

from src.aq import QuantizedWeight


torch.set_num_threads(16)
torch.backends.cudnn.allow_tf32 = False
torch.backends.cuda.matmul.allow_tf32 = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_loading_dir = '/extra_disk_1/vahe1994/BRRR/layer10.self_attn.q_proj.input_activation.pt'
num_codebooks = 1
nbits_per_codebook = 14
out_group_size = 1
in_group_size = 8
batch_size = 16384
beam_size = 1
beam_search_epochs = 100
sparsity_regularizer = 0
print_frequency = 10
scale_nbits = 2    # 0 means no scales, 16 means no compression;
codebook_values_nbits = 16  # less than 16 means we quantize codebooks as well
init_max_iter = 100

env: CUDA_VISIBLE_DEVICES=2
env: TRANSFORMERS_CACHE=/mnt/LLM/hub
env: OMP_NUM_THREADS=16


In [2]:
import wandb

os.environ["WANDB_NOTEBOOK_NAME"] = os.path.join(os.getcwd(), ipynbname.name() + ".ipynb")

# start a new wandb run to track this script
run = wandb.init(
    # set the wandb project where this run will be logged
    dir=os.getcwd(),
    project="AddQuantization-debug",
    entity = "rock-and-roll",
    save_code=True,
    name = f"{ipynbname.name()}_AQ_{num_codebooks=}_{out_group_size=}_{in_group_size=}_{nbits_per_codebook=}_{beam_search_epochs=}",
    settings=wandb.Settings(code_dir="."),
    # track hyperparameters and run metadata
    config={
    "num_codebooks" : num_codebooks,
    "out_group_size": out_group_size,
    "in_group_size": in_group_size,
    "group_size" : out_group_size * in_group_size,
    "batch_size" : batch_size,
    "beam_size" : beam_size,
    "nbits_per_codebook" : nbits_per_codebook,
    "codebook_values_nbits": codebook_values_nbits,
    "scale_nbits": scale_nbits,
    "beam_search_epochs": beam_search_epochs,
    "sparsity_regularizer": sparsity_regularizer,
    "init_max_iter": init_max_iter,
    }
)

wandb: Currently logged in as: justheuristic (rock-and-roll). Use `wandb login --relogin` to force relogin


wandb: WARNING No relevant files were detected in the specified directory. No code will be logged to your run.


In [3]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-70b-hf", torch_dtype='auto', low_cpu_mem_usage=True)

X = torch.load(input_loading_dir, map_location='cpu').float().flatten(0, -2)
reference_weight = model.model.layers[10].self_attn.q_proj.weight.detach().to(device).float()

XTX = torch.zeros(X.shape[-1], X.shape[-1], device=device, dtype=torch.float64)
for i in range(0, len(X), batch_size):
    x_batch = X[i: i + batch_size].cuda().double()
    XTX.addmm_(x_batch.T, x_batch, alpha=1/len(X))
    del x_batch
XTX = XTX.float()
del X

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [4]:
quantized_weight = QuantizedWeight(
    reference_weight=reference_weight, num_codebooks=num_codebooks,
    nbits_per_codebook=nbits_per_codebook, scale_nbits=scale_nbits,
    out_group_size=out_group_size, in_group_size=in_group_size,
    verbose=True, max_iter=init_max_iter,   # faster init, not tested
)
run.log({"Avg_bits": quantized_weight.estimate_nbits_per_parameter()})
opt = torch.optim.Adam(quantized_weight.parameters(), lr=1e-4, betas=(0.0, 0.95), amsgrad=True)

for epoch in range(1000):
    start = time.perf_counter()
    delta_weight = (quantized_weight() - reference_weight).double()
    loss = (delta_weight @ XTX.double()).flatten() @ delta_weight.flatten() / len(delta_weight)
    opt.zero_grad()
    loss.backward()
    opt.step()
    
    run.log({'loss':loss.item()}, step=epoch)
    
    if epoch % print_frequency == 0:
        print(f"loss={loss.item():.10f}\t",
              f"time_on_epoch {epoch} = {time.perf_counter() - start}")
    if (epoch + 1) % beam_search_epochs == 0:
        quantized_weight.beam_search_update_codes_(
            XTX, reference_weight, beam_size=beam_size, sparsity_regularizer=sparsity_regularizer,
            dim_rng=random.Random(), verbose=True)

        if sparsity_regularizer != 0:
            sparsity_rate = ((quantized_weight.codes == 0).sum() / quantized_weight.codes.numel()).item()
            print(f"Sparsity rate {sparsity_rate:.5f}")
            run.log({'sparsity rate': sparsity_rate}, step=epoch)
            mean_code_nbits = sum(get_mean_nbits_by_codebook(quantized_weight.codes)) / num_codebooks
            print(f"mean_code_nbits {mean_code_nbits:.5f}")
            run.log({'Mean codebook length nbits': mean_code_nbits}, step=epoch)
            if in_group_size > 1 and out_group_size > 1:
                curr_avg_bits  = calc_avg_bits(num_codebooks, 1, mean_code_nbits,
                                     nbits_per_codebook, in_features, out_features, scale_nbits)
                run.log({"Avg_bits": curr_avg_bits}, step=epoch)

/home/jheuristic/GPTAQ_tp/Notebooks/../src/kmeans.py:223: UserWarning: torch.searchsorted(): boundary tensor is non-contiguous, this will lower the performance due to extra data copy when converting non-contiguous tensor to contiguous, please use contiguous boundary tensor if possible. This message will only appear once per program. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/BucketizationUtils.h:38.)
  groupwise_cluster_indices = torch.searchsorted(border_indices[:, 1:], groupwise_ranks_1based, side='left')


initializing with kmeans:   0%|          | 0/1 [00:00<?, ?it/s]

/home/jheuristic/GPTAQ_tp/Notebooks/../src/aq.py:559: UserWarning: index_reduce() is in beta and the API may change at any time. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cuda/Indexing.cu:1193.)
  codebook_i, _, _ = fit_kmeans(


loss=0.0254404022	 time_on_epoch 0 = 1.802529029082507
loss=0.0210026318	 time_on_epoch 10 = 0.13315559295006096
loss=0.0175706957	 time_on_epoch 20 = 0.13393667386844754
loss=0.0149330052	 time_on_epoch 30 = 0.13344294298440218
loss=0.0129503049	 time_on_epoch 40 = 0.13311199308373034
loss=0.0114855729	 time_on_epoch 50 = 0.13305627298541367
loss=0.0104046396	 time_on_epoch 60 = 0.13350988295860589
loss=0.0096008013	 time_on_epoch 70 = 0.1330640739761293
loss=0.0089960344	 time_on_epoch 80 = 0.1331505631096661


wandb: WARNING Step only supports monotonically increasing values, use define_metric to set a custom x axis. For details see: https://wandb.me/define-metric
wandb: WARNING (User provided step: 0 is less than current step: 1. Dropping entry: {'loss': 0.025440402197482403, '_timestamp': 1702432730.4769356}).


...

  0%|          | 0/1024 [00:00<?, ?it/s]

loss=0.0022542918	 time_on_epoch 800 = 0.13398651196621358
loss=0.0022535495	 time_on_epoch 810 = 0.1330864131450653
loss=0.0022530730	 time_on_epoch 820 = 0.13310854299925268
loss=0.0022526800	 time_on_epoch 830 = 0.13312069419771433
loss=0.0022523345	 time_on_epoch 840 = 0.13311032392084599
loss=0.0022520199	 time_on_epoch 850 = 0.1331010339781642
loss=0.0022517274	 time_on_epoch 860 = 0.13319650408811867
loss=0.0022514521	 time_on_epoch 870 = 0.1330987939145416
loss=0.0022511904	 time_on_epoch 880 = 0.13309921487234533
loss=0.0022509403	 time_on_epoch 890 = 0.13314738497138023


  0%|          | 0/1024 [00:00<?, ?it/s]

loss=0.0022368615	 time_on_epoch 900 = 0.13390998705290258
loss=0.0022362412	 time_on_epoch 910 = 0.1330994670279324
loss=0.0022358361	 time_on_epoch 920 = 0.13309631682932377
loss=0.0022354991	 time_on_epoch 930 = 0.1331188678741455
loss=0.0022352007	 time_on_epoch 940 = 0.1332482979632914
loss=0.0022349277	 time_on_epoch 950 = 0.1331654479727149
loss=0.0022346728	 time_on_epoch 960 = 0.13312231819145381
loss=0.0022344319	 time_on_epoch 970 = 0.13314944808371365
loss=0.0022342022	 time_on_epoch 980 = 0.13314449903555214
loss=0.0022339821	 time_on_epoch 990 = 0.13316197786480188


  0%|          | 0/1024 [00:00<?, ?it/s]